<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%9A%D0%B0%D0%B7%D0%B0%D0%BD%D1%86%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import re
import pandas as pd

# Словарь предметов с вариантами названий
SUBJECTS = {
    "Математика": ["Математика"],
    "Русский язык": ["Русский язык"],
    "Литература": ["Литература"],
    "История": ["История"],
    "Обществознание": ["Обществознание"],
    "География": ["География"],
    "Биология": ["Биология"],
    "Химия": ["Химия"],
    "Физика": ["Физика"],
    "Информатика": ["Информатика"],
    "Иностранный язык": ["Английский язык", "Немецкий язык", "Французский язык", "Испанский язык"],
    "Музыка": ["Музыка"],
    "Изобразительное искусство": ["Изобразительное искусство", "ИЗО"],
    "Технология": ["Технология"],
    "ОБЖ": ["Основы безопасности жизнедеятельности", "ОБЖ"],
    "Физкультура": ["Физическая культура", "Физкультура"],
    "Алгебра": ["Алгебра"],
    "Окружающий мир": ["Окружающий мир"],
    "ОДНКР": ["ОДНКР"],
    "Экология": ["Экология"],
    "Ботаника": ["Ботаника"],
    "Естествознание": ["Естествознание"],
    "ОДНКНР": ["ОДНКНР"],
    "Кибербезопасность": ["Кибербезопасность"],
    "Геометрия": ["Геометрия"]
}

BASE_URL = "https://rulex.kpfu.ru/booklist"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Функция для получения списка ссылок на "Список терминов"
def get_term_links():
    response = requests.get(BASE_URL, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    links = [
        "https://rulex.kpfu.ru" + a["href"]
        for a in soup.find_all("a", string="Список терминов")
    ]
    return links

# Функция для извлечения предмета из названия учебника
def extract_subject(title):
    for subject, variations in SUBJECTS.items():
        if any(variant in title for variant in variations):
            return subject
    return title  # Если предмет не найден, возвращаем полный заголовок

# Функция для извлечения минимального класса из заголовка
def extract_min_class(title):
    """Извлекает минимальный класс из заголовка учебника."""
    matches = re.findall(r"(\d+)[-]?(?:й|го)?\s*кл[.]?", title, re.IGNORECASE)
    roman_matches = re.findall(r"\b[IVXLCDM]+\b\s*класс", title, re.IGNORECASE)

    classes = [int(match) for match in matches]

    # Обработка римских чисел (например, VII класс -> 7)
    roman_to_int = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "VI": 6, "VII": 7, "VIII": 8, "IX": 9, "X": 10}
    for match in roman_matches:
        roman_num = match.split()[0]
        if roman_num in roman_to_int:
            classes.append(roman_to_int[roman_num])

    return min(classes) if classes else None

# Функция для парсинга одной страницы
def parse_page(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    # Извлекаем заголовок учебника
    class_info = soup.find("h4").text.strip() if soup.find("h4") else ""
    subject = extract_subject(class_info)
    min_class = extract_min_class(class_info)

    # Извлекаем таблицу терминов
    table = soup.find("table", class_="w3-table-all")
    terms = []

    for row in table.find_all("tr")[1:]:  # Пропускаем заголовок
        cols = row.find_all("td")
        if len(cols) >= 1:
            term = cols[0].text.strip()
            terms.append((term, subject, min_class))

    return terms

# Основная функция
def main():
    term_links = get_term_links()
    all_terms = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(parse_page, term_links)

    for result in results:
        all_terms.extend(result)

    # Убираем дубликаты, оставляя минимальный класс
    term_dict = {}
    for term, subject, min_class in all_terms:
        if term in term_dict:
            existing_subject, existing_min_class = term_dict[term]
            if existing_subject == subject:
                if existing_min_class is None or min_class is None:
                    term_dict[term] = (subject, existing_min_class or min_class)
                else:
                    term_dict[term] = (subject, min(existing_min_class, min_class))
        else:
            term_dict[term] = (subject, min_class)

    # Создаем DataFrame и сохраняем в Excel
    df = pd.DataFrame([(k, v[0], v[1]) for k, v in term_dict.items()],
                      columns=["Слово или сочетание", "Предмет", "Минимальный класс, где встречается"])
    df.to_excel("terms.xlsx", index=False)
    print("Файл terms.xlsx успешно создан!")

if __name__ == "__main__":
    main()